# Making CSV file

In [ ]:
import os
import pandas as pd
import cv2
from deepface import DeepFace

from dominant_color_detection import detect_colors
from PIL import ImageColor

make dataframe whose column contains info. of IMGs

In [ ]:
album_data = pd.DataFrame(columns=['master_release', 'path','is_multi_labeled', \
    'histo_grey', 'histo_rgb', 'dominant_r', 'dominant_g', 'dominant_b', \
        'age', 'gender', 'race', 'dominant_race', 'emotion', 'dominant_emotion'])

In [ ]:
win_dir = r"D:\AlbumCover"
macos_dir = "/Users/cooky/Documents/GitHub/0220dbtest/img/"

read_path = win_dir

Make Genre Columns

In [ ]:
for dir in os.listdir(read_path):
    if dir == '.DS_Store':
        continue
    if dir == 'multi':
        continue
    album_data[dir] = 0

In [ ]:
# Read the names of folders in the path
for dir in os.listdir(read_path):
    
    if dir == 'multi' or dir == '.DS_Store':
        continue
    
    # open the each folder
    img_files = os.listdir(os.path.join(read_path, dir));
    if dir == '.DS_Store':
        continue
    
    # read the name of IMG files and put'em into dataframe
    for img_file in img_files:
        if img_file == '.DS_Store':
            continue
        img_path = os.path.join(os.path.join(dir, img_file))
        img_info = cv2.imread(os.path.join(read_path, img_path), 0)
        
        # check whether there's a face in the album cover
        try:
            obj = DeepFace.analyze(img_path=os.path.join(read_path, img_path), actions = ['age', 'gender', 'race', 'emotion'])
        except ValueError:
            obj = None
        finally:
            # 0427 dominant color
            colors, ratio = detect_colors(os.path.join(read_path, img_path), 1)
            rgb_tuple = ImageColor.getcolor(colors[0], "RGB")
            # -----
            
            if obj is not None:
                print('single yes')
                album_data = album_data.append({'master_release' : img_file[:-5],
                                                'path' : img_path,
                                                'is_multi_labeled' : False,
                                                'histo_grey' : cv2.calcHist([img_info], [0], None, [256], [0,256],),
                                                # 0427 dominant color
                                                'dominant_r' : rgb_tuple[0],
                                                'dominant_g' : rgb_tuple[1],
                                                'dominant_b' : rgb_tuple[2],
                                                # -----
                                                dir:1,
                                                'age' : obj['age'],
                                                'gender' : obj['gender'],
                                                'race' : str(obj['race']),
                                                'dominant_race' : obj['dominant_race'],
                                                'emotion' : str(obj['emotion']),
                                                'dominant_emotion' : obj['dominant_emotion']
                                                }, ignore_index=True)
            else:
                album_data = album_data.append({'master_release' : img_file[:-5],
                                                'path' : img_path,
                                                'is_multi_labeled' : False,
                                                'histo_grey' : cv2.calcHist([img_info], [0], None, [256], [0,256],),
                                                # 0427 dominant color
                                                'dominant_r' : rgb_tuple[0],
                                                'dominant_g' : rgb_tuple[1],
                                                'dominant_b' : rgb_tuple[2],
                                                # -----
                                                dir:1,
                                                }, ignore_index=True)


    

# Now adding multi genre album data

multi_path =  os.path.join(read_path, 'multi')

# each folders
for multi_gens in os.listdir(multi_path):
    if multi_gens == '.DS_Store':
        continue
    # each songs
    for multi_gen_songs in os.listdir(os.path.join(multi_path, multi_gens)):
        if multi_gen_songs == '.DS_Store':
            continue
        # make temp dataframe
        img_path = os.path.join(multi_path, multi_gens, multi_gen_songs)
        img_info = cv2.imread(img_path, 0)
        # 0427 dominant color
        colors, ratio = detect_colors(os.path.join(multi_path, multi_gens, multi_gen_songs), 1)
        rgb_tuple = ImageColor.getcolor(colors[0], "RGB")
        # -----
        imsi_df = pd.DataFrame(columns=['master_release', 'path','is_multi_labeled', \
                                        'histo_grey', 'histo_rgb', 'dominant_r', 'dominant_g', 'dominant_b' \
                                        ])
        print(multi_gen_songs)
        imsi_df = imsi_df.append({'master_release' : multi_gen_songs[:-5],
                                        'path' : os.path.join('multi', multi_gens, multi_gen_songs),
                                        'is_multi_labeled' : True,
                                        # 0427 dominant color
                                        'dominant_r' : rgb_tuple[0],
                                        'dominant_g' : rgb_tuple[1],
                                        'dominant_b' : rgb_tuple[2],
                                        # -----
                                        'histo_grey' : cv2.calcHist([img_info], [0], None, [256], [0,256])},
                                        ignore_index=True)
        
        try:
            obj = DeepFace.analyze(img_path=img_path, actions = ['age', 'gender', 'race', 'emotion'])
        except ValueError:
            obj = None
        finally:
        # adding genre info
            for gen in multi_gens.split('+'):
                imsi_df[gen] = 1
            
            if obj is not None:
                print('multi yes')
                for data in obj:
                    if data != 'region':
                        imsi_df[data] = str(obj[data])

            #adding imsi_df into album_data
            album_data = album_data.append(imsi_df, ignore_index=True)



In [ ]:
# fillNa
album_data = album_data.fillna(0)

Refining Little bit 

In [ ]:
album_data = album_data.drop(['Unnamed: 0', 'is_multi_labeled', 'histo_grey', 'histo_rgb', 'dominant_bage'], axis=1)

    
album_data = album_data.reindex(columns=['master_release', 'path', 'dominant_r', 'dominant_g', 'dominant_b', 'gender','age','dominant_race', 'emotion', 'Blues', 'Brass & Military', "Children's", \
    'Classical', 'Electronic', 'Folk, World, & Country', 'Funk&Soul', 'Hip Hop', 'Jazz', 'Latin', 'Non-Music', 'Pop', 'Reggae', 'Rock', 'Stage & Screen'])


Save without index

In [ ]:
pd.DataFrame.to_csv(album_data, 'album_data_0427.csv', index=False)